In [1]:
import requests,time,datetime
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import utils.DateAndTimeUtil as datu
import utils.CommonUtil as cu
import utils.StockSpider as ss

In [2]:
df_hsA = pd.read_excel("data/沪深A股.xls",converters= {u'StockCode':str})

In [3]:
df = df_hsA

In [5]:
# dir_path = 'E:/wangtao/PythonWorkSpace/SpiderSpace/Stock_Companies/data/'
# ss.getAnnualReports(dir_path,df,'2019')

ss.getStockDataByType('E:/wangtao/PythonWorkSpace/SpiderSpace/Stock_Companies/data/')
# ss.getStockDataByType('E:/PythonWorkSpace/DataAnalysis/Stock_Companies/data/')


正在获取：沪深A股...请勿关闭浏览器
第153页-第0条....

Traceback (most recent call last):
  File "E:\wangtao\PythonWorkSpace\SpiderSpace\Stock_Companies\utils\StockSpider.py", line 126, in getStockDataByType
    sheet.write(row_index,4,qrr.text)
  File "D:\Anaconda3\lib\site-packages\selenium\webdriver\remote\webelement.py", line 76, in text
    return self._execute(Command.GET_ELEMENT_TEXT)['value']
  File "D:\Anaconda3\lib\site-packages\selenium\webdriver\remote\webelement.py", line 633, in _execute
    return self._parent.execute(command, params)
  File "D:\Anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 321, in execute
    self.error_handler.check_response(response)
  File "D:\Anaconda3\lib\site-packages\selenium\webdriver\remote\errorhandler.py", line 242, in check_response
    raise exception_class(message, screen, stacktrace)
selenium.common.exceptions.StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=80.0.3987.132)



In [6]:
# 显示所有列
pd.set_option('display.max_columns', None)
# 显示所有行
pd.set_option('display.max_rows', None)

In [7]:
df_2014 = pd.read_excel("data/2014年报.xls",converters= {u'stock_code':str})
df_2015 = pd.read_excel("data/2015年报.xls",converters= {u'stock_code':str})
df_2016 = pd.read_excel("data/2016年报.xls",converters= {u'stock_code':str})
df_2017 = pd.read_excel("data/2017年报.xls",converters= {u'stock_code':str})
df_2018 = pd.read_excel("data/2018年报.xls",converters= {u'stock_code':str})
df_2019 = pd.read_excel("data/2019年报.xls",converters= {u'stock_code':str})

In [8]:
code_list = []
reports = [df_2014,df_2015,df_2016,df_2017,df_2018,df_2019]
for df_report in reports:
    codes = []
    for index, row in df_report.iterrows():
        # 净资产收益率大于15%
        if (row['stock_code'][0] != '3') & \
            (row['avg_roe'] > 0.15) & \
           (row['total_revenue_gr'] > 0) & \
            (row['basic_eps'] > 0) & \
            (row['np_per_share'] > 0) & \
            (row['operate_cash_flow_ps'] > 0):
#             (row['operate_cash_flow_ps_gr'] > 0):
            #满足条件的加入
            codes.append(row['stock_code'])
            
    code_list.append(codes)

In [9]:
intersection = code_list[0]
# 取每年的交集
for i in range(1,len(code_list)):
    intersection = list(set(intersection).intersection(set(code_list[i])))

In [10]:
df_target = df[df['StockCode'].apply(lambda x: (x in intersection))]

In [13]:
df_target.shape

(64, 19)

In [14]:
df_target = df_target[df_target['PE_D'].apply(lambda x:(x != '未公布') & (x != '0'))]
df_target = df_target[df_target['PE_S'].apply(lambda x:(x != '未公布') & (x != '0'))]
# 新增市盈率中位数
df_target['PE_M'] = df_target['StockCode'].apply(lambda x:ss.getPEMedian(x))
df_target = df_target[df_target['PE_M'] != 0]

df_target['pe_rate'] = df_target.apply(lambda x:(float(x['PE_D']) / x['PE_M']),axis=1)
df_target = df_target[df_target['pe_rate'] < 0.6]

In [15]:
df_target.shape

(4, 21)

In [13]:
df_target.sort_values(by=['pe_rate'],ascending=True)

,StockCode,StockName,Price,PriceLimit,QuantityRelativeR,TurnoverRate,Trade,PB,PE_S,PE_D,EarningsPerShare,NetProfitDes,ROE,GrossProfitRate,NetAssetValuePerShare,CapitalStock,ScaleShareType,FinanceAnalize,MoneyFlowPerShare,PE_M,pe_rate
578,600566,济川药业,23.79,1.06%,0.81,0.81%,医药生物 — 中药Ⅱ,3.36,11.37,11.020,1.60元,13.06亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比下降0.24%\t...,23.05%,84.45%,7.00元,8.15亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t大盘股\t\t\t\t\t\t\t\...,"三线蓝筹,白马股,绩优股,",2.19元,27.620,0.398986
1788,000963,华东医药,19.85,-1.10%,0.53,0.69%,医药生物 — 化学制药,3.01,15.49,11.901,1.26元,22.13亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比增长22.33%\...,20.14%,32.34%,6.66元,17.50亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t大盘股\t\t\t\t\t\t\t\...,"三线蓝筹,",0.52元,28.090,0.423674
1463,002572,索菲亚,20.21,-0.69%,0.83,2.01%,轻工制造 — 家用轻工,3.43,17.47,17.472,1.17元,10.70亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比增长11.56%\...,19.68%,未公布,5.96元,9.12亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t中盘股\t\t\t\t\t\t\t\...,"绩优股,",未公布,39.035,0.447598
3181,002262,恩华药业,11.97,-3.31%,0.73,2.06%,医药生物 — 化学制药,3.49,19.04,19.037,0.66元,6.63亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比增长26.32%\t...,18.32%,未公布,3.55元,10.19亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t中盘股\t\t\t\t\t\t\t\...,"绩优股,",未公布,40.890,0.465566
706,603886,元祖股份,17.33,0.70%,0.78,1.83%,食品饮料 — 食品加工,2.72,17.09,11.251,1.15元,2.75亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比增长6.94%\t\...,18.56%,65.91%,6.33元,2.40亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t小盘股\t\t\t\t\t\t\t\...,"绩优股,",2.64元,22.010,0.511177
1189,600398,海澜之家,6.92,-0.29%,0.75,0.20%,纺织服装 — 服装家纺,2.43,8.93,8.844,0.59元,26.16亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比下降0.45%\t...,19.85%,41.77%,2.86元,44.20亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t大盘股\t\t\t\t\t\t\t\...,"三线蓝筹,绩优股,",0.05元,15.045,0.587836
2403,600388,龙净环保,11.03,-1.87%,0.57,1.04%,机械设备 — 专用设备,2.07,13.70,13.696,0.82元,8.77亿元\n\t\t\t\t\t\t\n\t\t\t\t\t\t同比增长9.5%\t\t...,15.09%,未公布,5.44元,10.69亿股,\n\t\t\t\t\t\t\t\t\t\t\t\t\t中盘股\t\t\t\t\t\t\t\...,NaN,未公布,22.880,0.598601


In [57]:
df_target.shape

(98, 20)